In [21]:
# import modules 
import os 
import quantstats as qs 
import pandas as pd 
import yfinance as yf
import warnings
%matplotlib inline
warnings.simplefilter("ignore") 
import numpy as np

In [22]:
# Function to calculate pivot point
def calculate_pivot_point(high, low, close):
    pivot = (high + low + close) / 3
    return pivot

# Function to check if volume is high (above average)
def is_volume_high(volume, avg_volume):
    return volume > avg_volume

# Function to calculate ADOR (Average Daily Opening Range)
def calculate_ador(df, window=20):
    opening_range = df['High'].rolling(4).max() - df['Low'].rolling(4).min()
    ador = opening_range.rolling(window).mean()
    return ador

# Function to implement the trading strategy
def trading_strategy(df, is_forex=True):
    signals = pd.Series(index=df.index, dtype=int)
    in_position = False
    
    for i in range(4, len(df)):
        if not in_position:
            # Short order conditions
            if (is_volume_high(df['Volume'].iloc[i], df['Volume'].rolling(20).mean().iloc[i]) and
                df['Close'].iloc[i] < calculate_pivot_point(df['High'].iloc[i-1], df['Low'].iloc[i-1], df['Close'].iloc[i-1]) and
                df['Close'].iloc[i] < df['Close'].iloc[i-1]):
                signals.iloc[i] = -1
                in_position = True
            
            # Buy order conditions
            elif (is_volume_high(df['Volume'].iloc[i], df['Volume'].rolling(20).mean().iloc[i]) and
                  df['High'].iloc[i] > df['High'].iloc[i-1]):
                signals.iloc[i] = 1
                in_position = True
            
            # Buy order based on ADOR
            elif df['ADOR'].iloc[i] > (df['High'].iloc[i] - df['Low'].iloc[i]):
                signals.iloc[i] = 1
                in_position = True
            
            # Sell order based on ADOR
            elif (df['ADOR'].iloc[i] < (df['High'].iloc[i] - df['Low'].iloc[i]) and
                  df['Close'].iloc[i] < df['Close'].iloc[i-1]):
                signals.iloc[i] = -1
                in_position = True
        
        else:
            # Check for stop loss or take profit
            if is_forex:
                if signals.iloc[i-1] == 1:  # Long position
                    if df['Low'].iloc[i] <= df['Open'].iloc[i-1] - 0.001:  # 10 pip stop loss
                        signals.iloc[i] = 0
                        in_position = False
                    elif df['High'].iloc[i] >= df['Open'].iloc[i-1] + 0.0025:  # 25 pip take profit
                        signals.iloc[i] = 0
                        in_position = False
                elif signals.iloc[i-1] == -1:  # Short position
                    if df['High'].iloc[i] >= df['Open'].iloc[i-1] + 0.001:  # 10 pip stop loss
                        signals.iloc[i] = 0
                        in_position = False
                    elif df['Low'].iloc[i] <= df['Open'].iloc[i-1] - 0.0025:  # 25 pip take profit
                        signals.iloc[i] = 0
                        in_position = False
            else:  # Micro futures
                if signals.iloc[i-1] == 1:  # Long position
                    if df['Low'].iloc[i] <= df['Open'].iloc[i-1] - 24:  # 24 point stop loss
                        signals.iloc[i] = 0
                        in_position = False
                    elif df['High'].iloc[i] >= df['Open'].iloc[i-1] + 45:  # 45 point take profit
                        signals.iloc[i] = 0
                        in_position = False
                elif signals.iloc[i-1] == -1:  # Short position
                    if df['High'].iloc[i] >= df['Open'].iloc[i-1] + 24:  # 24 point stop loss
                        signals.iloc[i] = 0
                        in_position = False
                    elif df['Low'].iloc[i] <= df['Open'].iloc[i-1] - 45:  # 45 point take profit
                        signals.iloc[i] = 0
                        in_position = False
    
    return signals


In [23]:


# Download data
usdjpy = yf.Ticker("USDJPY=X")
mnqu4 = yf.Ticker("MNQ=F")  # Assuming MNQ=F is the correct ticker for mnqu4

usdjpy_data = usdjpy.history(period="5d", interval="15m")
mnqu4_data = mnqu4.history(period="5d", interval="15m")

# Calculate ADOR for both instruments
usdjpy_data['ADOR'] = calculate_ador(usdjpy_data)
mnqu4_data['ADOR'] = calculate_ador(mnqu4_data)

# Apply trading strategy
usdjpy_signals = trading_strategy(usdjpy_data, is_forex=True)
mnqu4_signals = trading_strategy(mnqu4_data, is_forex=False)

# Calculate returns
usdjpy_returns = usdjpy_data['Close'].pct_change()
mnqu4_returns = mnqu4_data['Close'].pct_change()

usdjpy_strategy_returns = usdjpy_returns * usdjpy_signals.shift(1)
mnqu4_strategy_returns = mnqu4_returns * mnqu4_signals.shift(1)

# Ensure the index is DatetimeIndex
usdjpy_strategy_returns.index = pd.to_datetime(usdjpy_strategy_returns.index)
mnqu4_strategy_returns.index = pd.to_datetime(mnqu4_strategy_returns.index)

# Generate reports using Quantstats
#qs.reports.html(usdjpy_strategy_returns, output='usdjpy_backtest_report.html', title='USDJPY Backtest Report')
qs.reports.html(mnqu4_strategy_returns, output='mnqu4_backtest_report.html', title='MNQU4 Backtest Report')

print("Backtest completed. Reports generated: usdjpy_backtest_report.html and mnqu4_backtest_report.html")

ZeroDivisionError: float division by zero